In [1]:
from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator
import pandas as pd
import cobra
from mewpy.simulation import set_default_solver
set_default_solver('gurobi')

In [2]:
%store -r RPMI1640_plus_2mM_glutamine #set manually
media = RPMI1640_plus_2mM_glutamine
%store -r media_only_essential_reactions #calculated in separate notebook using essential_reactions functions from mewpy.simulation package
%store -r media_only_essential_genes #calculated in separate notebook using essential_genes functions from mewpy.simulation package
%store -r t_2020_4 #CCLE 2020 gene expression data formatted for input
expression = t_2020_4
%store -r c_dict #CCLE dataset for correlation between gene and protein expression data
%store -r proteomics_n15_scaled #CCLE 2020 protein expression data formatted for input, and scaled so bottom 50% of values are within 0 and 1.
p = proteomics_n15_scaled
p = p.transpose()

c = 'IGROV1' #set manually

for n in range(len(expression.index.to_list())):
    if (expression.index.to_list()[n]) == c: #iterate through gene expression dataframe to find target cell line
        row_number_transcriptomics = n
        
for n in range(len(p.index.to_list())):
    if (p.index.to_list()[n]) == c: #iterate through protein expression dataframe to find target cell line
        row_number_proteomics = n
        
experimental_growth_rate = 0.0376 #set manually

no stored variable or alias #set
no stored variable or alias manually


In [3]:
#load Human1 model and solve FBA optimised for biomass production, with media conditions only as constraint
model = read_sbml_model('/Users/kmeeson/Dropbox (The University of Manchester)/project_work_2023/Chapter 3 comparing algorithms/omics/data/Human-GEM-annotated.xml')
media_only_simulation = get_simulator(model, envcond = media)
media_only_result = media_only_simulation.simulate()
media_only_result.find(['biomass_human'])

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


,Flux rate
Reaction ID,
biomass_human,0.055515


In [5]:
#create dictionary containing expression values
#for non-Human models, 'ENS' term in the generateion of g_ensembl list must be changed to be organism-specific
t = expression
gene_exp = t.iloc[row_number_transcriptomics,:]

g_list = []
for k in gene_exp.keys():
    g_list.append(k)

g_ensembl = []
for n in range(len(g_list)):
    if 'ENS' not in (g_list[n].split())[0]:
        g_ensembl.append((g_list[n].split())[1])
    if 'ENS' in (g_list[n].split())[0]:
        g_ensembl.append((g_list[n].split())[0])
        
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
g_ensembl_2 = []
for n in range(len(g_ensembl)):
    for x in g_ensembl[n]:
        if x in punc:
            g_ensembl[n] = g_ensembl[n].replace(x, "")
    g_ensembl_2.append(g_ensembl[n])
g_ensembl_2

vs = []
for v in gene_exp:
    vs.append(v)
    
gene_exp_dict = {}
for n in range(len(g_ensembl_2)):
    gene_exp_dict[g_ensembl_2[n]] = vs[n]
    
prot_exp = p.iloc[row_number_proteomics,:]

p_list = []
for k in prot_exp.keys():
    p_list.append(k)

vs = []
for v in prot_exp:
    vs.append(v)
    
p_dict = {}
for n in range(len(p_list)):
    p_dict[p_list[n]] = vs[n]
    
in_both_well_correlated = {}
for k,v in gene_exp_dict.items():
    if k in p_dict.keys():
        if k in c_dict.keys():
            in_both_well_correlated[k] = ((gene_exp_dict[k]+p_dict[k])/2)
print('number of genes in both datasets, and well correlated:',len(in_both_well_correlated))

ANDs = []
ORs = []
ANDORs = []
one_gene = []
no_gene = []

for r in model.reactions:
    if 'and' in r.gene_reaction_rule and 'or' not in r.gene_reaction_rule:
        ANDs.append(r.id)
    if 'and' in r.gene_reaction_rule and 'or' in r.gene_reaction_rule:
        ANDORs.append(r.id)
    if 'or' in r.gene_reaction_rule and 'and' not in r.gene_reaction_rule:
        ORs.append(r.id)
    if len(r.gene_reaction_rule) == 0:
        no_gene.append(r.id)
    if len(r.gene_reaction_rule) != 0:
        if 'or' in r.gene_reaction_rule:
            continue
        elif 'and' in r.gene_reaction_rule:
            continue
        else:
            one_gene.append(r.id)

print('AND rules: ', len(ANDs))
print('ANDOR rules: ', len(ANDORs))
print('OR rules: ', len(ORs))
print('ONE GENE rules: ', len(one_gene))
print('NO GENE rules: ', len(no_gene))
print('Proportion of model not annotated: ', len(no_gene)/len(model.reactions)*100, '%')
print('Proportion of model which IS annotated: ', 100-(len(no_gene)/len(model.reactions)*100), '%')
print('Total Reactions = ', len(model.reactions))
print(len(ORs)+len(ANDs)+len(ANDORs)+len(one_gene)+len(no_gene))

one_gene_forward = []
one_gene_reversible = []
for r in one_gene:
    if model.reactions.get_by_id(r).reversibility == True:
        one_gene_reversible.append(r)
    else:
        one_gene_forward.append(r)
print('number of reversible, one gene reactions =', len(one_gene_reversible))
print('number of forward, one gene reactions =', len(one_gene_forward))

number of genes in both datasets, and well correlated: 3260
AND rules:  653
ANDOR rules:  129
OR rules:  3972
ONE GENE rules:  3282
NO GENE rules:  5060
Proportion of model not annotated:  38.63775198533904 %
Proportion of model which IS annotated:  61.36224801466096 %
Total Reactions =  13096
13096
number of reversible, one gene reactions = 1074
number of forward, one gene reactions = 2208


In [6]:
#to make loops in below cells more efficient...
#add expression constraints, if too tight, then relax bounds and resolve
def check_forward_constraints(reaction_id, upper_bound, exp_growth, to_print, printResult = True):
    if printResult == True:
        model.reactions.get_by_id(reaction_id).bounds = (0,upper_bound)
        solution = model.optimize()
        if solution.fluxes['biomass_human'] < exp_growth:
            print(reaction_id, ':', 'constrained bounds', model.reactions.get_by_id(reaction_id).bounds, solution.fluxes['biomass_human'])
            model.reactions.get_by_id(reaction_id).bounds = (0,1000)
            solution = model.optimize()
            print(r, ':', 're-opened bounds', model.reactions.get_by_id(reaction_id).bounds, solution.fluxes['biomass_human'])
            one_gene_constrained_reactions[reaction_id] = model.reactions.get_by_id(reaction_id).bounds 
        if solution.fluxes['biomass_human'] !=0:
            if solution.fluxes['biomass_human'] > exp_growth:
                print(reaction_id, to_print, model.reactions.get_by_id(reaction_id).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[reaction_id] = model.reactions.get_by_id(reaction_id).bounds
        return(model.reactions.get_by_id(reaction_id).bounds)
    
def check_reverse_constraints(reaction_id, upper_bound, exp_growth, to_print, printResult = True):
    if printResult == True:
        model.reactions.get_by_id(reaction_id).bounds = (-1*upper_bound,upper_bound)
        solution = model.optimize()
        if solution.fluxes['biomass_human'] < exp_growth:
            print(reaction_id, ':', 'constrained bounds', model.reactions.get_by_id(reaction_id).bounds, solution.fluxes['biomass_human'])
            model.reactions.get_by_id(reaction_id).bounds = (-1000,1000)
            solution = model.optimize()
            print(r, ':', 're-opened bounds', model.reactions.get_by_id(reaction_id).bounds, solution.fluxes['biomass_human'])
            one_gene_constrained_reactions[reaction_id] = model.reactions.get_by_id(reaction_id).bounds 
        if solution.fluxes['biomass_human'] !=0:
            if solution.fluxes['biomass_human'] > exp_growth:
                print(reaction_id, to_print, model.reactions.get_by_id(reaction_id).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[reaction_id] = model.reactions.get_by_id(reaction_id).bounds
        return(model.reactions.get_by_id(reaction_id).bounds)

In [7]:
one_gene_constrained_reactions = {}

for r in one_gene_forward:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            if model.reactions.get_by_id(r).gene_reaction_rule not in media_only_essential_genes:
                if model.reactions.get_by_id(r).gene_reaction_rule in in_both_well_correlated.keys():
                    if in_both_well_correlated[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                        one_gene_constrained_reactions[r] = check_forward_constraints(r, in_both_well_correlated[model.reactions.get_by_id(r).gene_reaction_rule], experimental_growth_rate, 'avg p&t bounds:', printResult = True)
                if model.reactions.get_by_id(r).gene_reaction_rule not in in_both_well_correlated.keys():
                    if model.reactions.get_by_id(r).gene_reaction_rule in p_dict.keys():
                        if p_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                            one_gene_constrained_reactions[r] = check_forward_constraints(r, p_dict[model.reactions.get_by_id(r).gene_reaction_rule], experimental_growth_rate, 'p bounds:', printResult = True)  
                    if model.reactions.get_by_id(r).gene_reaction_rule not in p_dict.keys():
                        if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys():
                            if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                                one_gene_constrained_reactions[r] = check_forward_constraints(r, gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule], experimental_growth_rate, 't bounds:', printResult = True) 
one_gene_forward_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
one_gene_forward_result = one_gene_forward_simulation.simulate()
print((one_gene_forward_result.find(['biomass_human'])).iloc[0,0])

one_gene_forward_essential_reactions = media_only_essential_reactions
one_gene_forward_essential_genes = media_only_essential_genes

for r in one_gene_reversible:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            if model.reactions.get_by_id(r).gene_reaction_rule not in media_only_essential_genes:
                if model.reactions.get_by_id(r).gene_reaction_rule in in_both_well_correlated.keys():
                    if in_both_well_correlated[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                        one_gene_constrained_reactions[r] = check_reverse_constraints(r, in_both_well_correlated[model.reactions.get_by_id(r).gene_reaction_rule], experimental_growth_rate, 'avg p&t bounds:', printResult = True)
                if model.reactions.get_by_id(r).gene_reaction_rule not in in_both_well_correlated.keys():
                    if model.reactions.get_by_id(r).gene_reaction_rule in p_dict.keys():
                        if p_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                            one_gene_constrained_reactions[r] = check_reverse_constraints(r, p_dict[model.reactions.get_by_id(r).gene_reaction_rule], experimental_growth_rate, 'p bounds:', printResult = True)  
                    if model.reactions.get_by_id(r).gene_reaction_rule not in p_dict.keys():
                        if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys():
                            if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                                one_gene_constrained_reactions[r] = check_reverse_constraints(r, gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule], experimental_growth_rate, 't bounds:', printResult = True)

all_one_gene_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
all_one_gene_result = all_one_gene_simulation.simulate()
print((all_one_gene_result.find(['biomass_human'])).iloc[0,0])

or_forward = []
or_reversible = []
for r in ORs:
    if model.reactions.get_by_id(r).reversibility == True:
        or_reversible.append(r)
    else:
        or_forward.append(r)
print('number of reversible, OR reactions =', len(or_reversible))
print('number of forward, OR reactions =', len(or_forward))

for r in or_forward:
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions:
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_both = []
            genes_in_p = []
            genes_in_t = []
            for gene in rule_genes:
                if gene in in_both_well_correlated.keys():
                    genes_in_both.append(gene)
                if gene not in in_both_well_correlated.keys():
                    if gene in p_dict.keys():
                        genes_in_p.append(gene)
                    if gene not in p_dict.keys():
                        if gene in gene_exp_dict.keys():
                            genes_in_t.append(gene)
            exp_list = []        
            for g in genes_in_both:
                exp_list.append(in_both_well_correlated[g])
            for g1 in genes_in_p:
                exp_list.append(p_dict[g1])
            for g2 in genes_in_t:
                exp_list.append(gene_exp_dict[g2])
            sum_of_expressions = sum(exp_list)
            #set bounds according to sum of expressions
            one_gene_constrained_reactions[r] = check_forward_constraints(r, sum_of_expressions, experimental_growth_rate, 'combined bounds:', printResult = True)

or_forward_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
or_forward_result = or_forward_simulation.simulate()
print((or_forward_result.find(['biomass_human'])).iloc[0,0])

for r in or_reversible:
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions:
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_both = []
            genes_in_p = []
            genes_in_t = []
            for gene in rule_genes:
                if gene in in_both_well_correlated.keys():
                    genes_in_both.append(gene)
                if gene not in in_both_well_correlated.keys():
                    if gene in p_dict.keys():
                        genes_in_p.append(gene)
                    if gene not in p_dict.keys():
                        if gene in gene_exp_dict.keys():
                            genes_in_t.append(gene)
            exp_list = []        
            for g in genes_in_both:
                exp_list.append(in_both_well_correlated[g])
            for g1 in genes_in_p:
                exp_list.append(p_dict[g1])
            for g2 in genes_in_t:
                exp_list.append(gene_exp_dict[g2])
            sum_of_expressions = sum(exp_list)
            #set bounds according to sum of expressions
            one_gene_constrained_reactions[r] = check_reverse_constraints(r, sum_of_expressions, experimental_growth_rate, 'combined bounds:', printResult = True)

all_or_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
all_or_result = all_or_simulation.simulate()
print((all_or_result.find(['biomass_human'])).iloc[0,0])

and_forward = []
and_reversible = []
for r in ANDs:
    if model.reactions.get_by_id(r).reversibility == True:
        and_reversible.append(r)
    else:
        and_forward.append(r)
print('number of reversible, AND reactions =', len(and_reversible))
print('number of forward, AND reactions =', len(and_forward))

for r in and_forward:
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions:
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            if len(exp_list) > 1:
                exp_list.sort()
                min_expression = exp_list[0]
            if len(exp_list) == 1:
                min_expression = exp_list[0]
            #set bounds according to sum of expressions
            one_gene_constrained_reactions[r] = check_forward_constraints(r, min_expression, experimental_growth_rate, 'combined bounds:', printResult = True)
            
and_forward_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
and_forward_result = and_forward_simulation.simulate()
print((and_forward_result.find(['biomass_human'])).iloc[0,0])

for r in and_reversible:
    if r not in media.keys():
        if r not in one_gene_forward_essential_reactions:
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            if len(exp_list) > 1:
                exp_list.sort()
                min_expression = exp_list[0]
            if len(exp_list) == 1:
                min_expression = exp_list[0]
            #set bounds according to sum of expressions
            one_gene_constrained_reactions[r] = check_reverse_constraints(r, min_expression, experimental_growth_rate, 'combined bounds:', printResult = True)
            
all_and_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
all_and_result = all_and_simulation.simulate()
print((all_and_result.find(['biomass_human'])).iloc[0,0])

HMR_3907 avg p&t bounds: (0, 3.794702675547022) 0.055514528776719314
HMR_4097 avg p&t bounds: (0, 2.675518549985725) 0.055514528776719314
HMR_4108 avg p&t bounds: (0, 2.675518549985725) 0.055514528776719314
HMR_4133 avg p&t bounds: (0, 2.675518549985725) 0.055514528776719314
HMR_4360 t bounds: (0, 2.1342209397606338) 0.055514528776719314
HMR_4372 avg p&t bounds: (0, 2.5223074461151924) 0.055514528776719314
HMR_7747 avg p&t bounds: (0, 3.272311883860553) 0.055514528776719314
HMR_8360 avg p&t bounds: (0, 3.3635783750824464) 0.055514528776719314
HMR_8757 t bounds: (0, 0.2016338611696504) 0.055514528776719314
HMR_5397 avg p&t bounds: (0, 2.420758723572405) 0.055514528776719314
HMR_5399 p bounds: (0, 0.9717719587825723) 0.055514528776719314
HMR_5400 p bounds: (0, 0.9717719587825723) 0.055514528776719314
HMR_8592 avg p&t bounds: (0, 2.626504618337019) 0.055514528776719314
HMR_8589 avg p&t bounds: (0, 2.626504618337019) 0.055514528776719314
HMR_8583 t bounds: (0, 0.0) 0.055514528776719314
HMR

HMR_4191 t bounds: (0, 1.2016338611696504) 0.055514528776719314
HMR_4212 t bounds: (0, 7.948075158491082) 0.055514528776719314
HMR_4422 t bounds: (0, 7.948075158491082) 0.055514528776719314
HMR_4424 avg p&t bounds: (0, 0.5447435390356505) 0.055514528776719314
HMR_4606 t bounds: (0, 0.5753123306874368) 0.055514528776719314
HMR_4784 avg p&t bounds: (0, 2.41200540252997) 0.055514528776719314
HMR_4785 avg p&t bounds: (0, 2.41200540252997) 0.055514528776719314
HMR_4787 avg p&t bounds: (0, 2.219196147972916) 0.055514528776719314
HMR_5384 avg p&t bounds: (0, 2.8229165817435473) 0.055514528776719314
HMR_8098 avg p&t bounds: (0, 3.569423758217926) 0.055514528776719314
HMR_8416 t bounds: (0, 1.372952097911829) 0.055514528776719314
HMR_8432 avg p&t bounds: (0, 0.7610110076436604) 0.055514528776719314
HMR_8608 t bounds: (0, 0.2016338611696504) 0.055514528776719314
HMR_8610 t bounds: (0, 0.0565835283663675) 0.055514528776719314
HMR_0457 t bounds: (0, 2.62760683812965) 0.055514528776719314
HMR_0460 

HMR_8683 avg p&t bounds: (0, 2.299410264203567) 0.055514528776719314
HMR_3868 avg p&t bounds: (0, 3.628235558961322) 0.055514528776719314
HMR_3869 avg p&t bounds: (0, 3.628235558961322) 0.055514528776719314
HMR_3889 avg p&t bounds: (0, 2.2253096466867577) 0.055514528776719314
HMR_3937 avg p&t bounds: (0, 4.048191855245717) 0.055514528776719314
HMR_4078 t bounds: (0, 5.464668267003445) 0.055514528776719314
HMR_4347 t bounds: (0, 0.0) 0.055514528776719314
HMR_4425 t bounds: (0, 0.5753123306874368) 0.055514528776719314
HMR_7135 avg p&t bounds: (0, 4.011829674754158) 0.055514528776719314
HMR_7141 p bounds: (0, 1.0004863248888411) 0.055514528776719314
HMR_7647 t bounds: (0, 0.2016338611696504) 0.055514528776719314
HMR_8689 t bounds: (0, 0.2016338611696504) 0.055514528776719314
HMR_8690 t bounds: (0, 0.2016338611696504) 0.055514528776719314
HMR_3755 avg p&t bounds: (0, 2.528640021349394) 0.055514528776719314
HMR_3763 avg p&t bounds: (0, 1.5172212150627313) 0.055514528776719314
HMR_3790 t bou

HMR_2238 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2239 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2240 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2241 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2242 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2243 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2244 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2245 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2246 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2247 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2249 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2250 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2251 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2252 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2254 p bounds: (0, 0.9707141727166014) 0.05551452877671937
HMR_2255 p bounds: (0, 0.9707141727166014) 0.0555145287

HMR_1992 t bounds: (0, 0.0976107966264223) 0.05551452877671943
HMR_2005 t bounds: (0, 0.0976107966264223) 0.05551452877671943
HMR_2007 t bounds: (0, 0.0976107966264223) 0.05551452877671943
HMR_2010 t bounds: (0, 0.0) 0.05551452877671943
HMR_2011 t bounds: (0, 0.0) 0.05551452877671943
HMR_6793 t bounds: (0, 0.1890338243900171) 0.05551452877671943
HMR_6794 t bounds: (0, 0.1890338243900171) 0.05551452877671943
HMR_6795 t bounds: (0, 0.1890338243900171) 0.05551452877671943
HMR_6796 t bounds: (0, 0.1890338243900171) 0.05551452877671943
HMR_7928 t bounds: (0, 0.0976107966264223) 0.05551452877671943
HMR_7932 t bounds: (0, 0.0976107966264223) 0.05551452877671943
HMR_7934 t bounds: (0, 0.0) 0.05551452877671943
HMR_7935 t bounds: (0, 0.2265085298086797) 0.05551452877671943
HMR_7936 t bounds: (0, 0.01435529297707) 0.05551452877671943
HMR_7937 t bounds: (0, 0.01435529297707) 0.05551452877671943
HMR_7938 t bounds: (0, 0.01435529297707) 0.05551452877671943
HMR_7945 t bounds: (0, 2.356143810225275) 0

...

HMR_3370 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3372 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3373 combined bounds: (0, 4.725195817226664) 0.03907215171758935
HMR_3501 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3503 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3505 combined bounds: (0, 4.725195817226664) 0.03907215171758935
HMR_3508 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3509 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3510 combined bounds: (0, 4.725195817226664) 0.03907215171758935
HMR_3513 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3514 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3515 combined bounds: (0, 4.725195817226664) 0.03907215171758935
HMR_3302 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3304 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3305 combined bounds

HMR_3286 combined bounds: (0, 4.339137384919585) 0.03907215171758935
HMR_3287 combined bounds: (0, 5.792334805706692) 0.03907215171758935
HMR_3290 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_3292 combined bounds: (0, 4.339137384919585) 0.03907215171758935
HMR_3293 combined bounds: (0, 5.792334805706692) 0.03907215171758935
HMR_1174 combined bounds: (0, 5.792334805706692) 0.03907215171758935
HMR_1177 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_1178 combined bounds: (0, 4.339137384919585) 0.03907215171758935
HMR_1179 combined bounds: (0, 5.792334805706692) 0.03907215171758935
HMR_1182 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_1183 combined bounds: (0, 4.339137384919585) 0.03907215171758935
HMR_1184 combined bounds: (0, 5.792334805706692) 0.03907215171758935
HMR_1216 combined bounds: (0, 5.792334805706692) 0.03907215171758935
HMR_1224 combined bounds: (0, 1.3950627995175775) 0.03907215171758935
HMR_1225 combined bounds: (0, 

HMR_5946 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5947 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5948 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5949 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5950 combined bounds: (0, 0.2016338611696504) 0.03907215167336077
HMR_5951 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5952 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5953 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5954 combined bounds: (0, 0.2016338611696504) 0.0390721516733552
HMR_5955 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5956 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5957 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5958 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5959 combined bounds: (0, 0.2016338611696504) 0.039072151673365633
HMR_5960 

FAOXC183C163m combined bounds: (0, 5.792334805706692) 0.03907215167333571
FAOXC184C163m combined bounds: (0, 5.792334805706692) 0.03907215167333571
FAOXC184C163x combined bounds: (0, 1.3950627995175775) 0.03907215167333571
FAOXC184C164m combined bounds: (0, 5.792334805706692) 0.03907215167333571
FAOXC184C164x combined bounds: (0, 1.3950627995175775) 0.03907215167333571
FAOXC185C164m combined bounds: (0, 5.792334805706692) 0.03907215167333571
FAOXC18C18OHm combined bounds: (0, 6.408542042296858) 0.03907215167333571
FAOXC201C181x combined bounds: (0, 1.3950627995175775) 0.03907215167333571
FAOXC204C184m combined bounds: (0, 5.792334805706692) 0.03907215167333571
FAOXC205C184x combined bounds: (0, 1.3950627995175775) 0.03907215167333571
FAOXC205C185m combined bounds: (0, 5.792334805706692) 0.03907215167333571
FAOXC221C201x combined bounds: (0, 1.3950627995175775) 0.03907215167333571
FAOXC225C204m combined bounds: (0, 5.792334805706692) 0.03907215167333571
FAOXC225C204x combined bounds: (0

HMR_6070 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6071 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6072 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6073 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6074 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6075 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6076 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6077 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6078 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6079 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6080 combined bounds: (-0.0976107966264223, 0.0976107966264223) 0.0390721511869495
HMR_6081 combined bounds: (-0.0976107966264

In [8]:
print('experimental doubling time: ',1/experimental_growth_rate)
print('model-predicted doubling time: ',1/(all_and_result.find(['biomass_human'])).iloc[0,0])

experimental doubling time:  26.595744680851062
model-predicted doubling time:  25.593676386657737


In [9]:
#store dataframe of fluxes for comparison to other models
#may aslo want to store copy of model with constraints for future use to avoid running above code
igrov1_t_2020_p_scaled = all_and_result.dataframe
%store igrov1_t_2020_p_scaled

Stored 'igrov1_t_2020_p_scaled' (DataFrame)
